In [1]:
!pip install -q transformers[sentencepiece] sacrebleu
!pip install -q Datasets

     |████████████████████████████████| 2.5MB 27.5MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 3.3MB 38.0MB/s 
     |████████████████████████████████| 901kB 43.7MB/s 
     |████████████████████████████████| 1.1MB 43.6MB/s 
     |████████████████████████████████| 266kB 30.5MB/s 
     |████████████████████████████████| 245kB 43.9MB/s 
     |████████████████████████████████| 122kB 46.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 645kB 35.8MB/s 
     |████████████████████████████████| 829kB 39.8MB/s 
     |████████████████████████████████| 10.6MB 40.9MB/s 
     |████████████████████████████████| 235kB 33.6MB/s 
     |████████████████████████████████| 1.3MB 41.8MB/s 
     |████████████████████████████████| 296kB 48.8MB/s 
     |████████████████████████████████| 143kB 50.9MB/s 
ERROR: tensorflow 2.5.0 has require

In [2]:
!gdown https://drive.google.com/uc?id=1vrfoukPD0GcoLaP8RewHvIKzcL-8Pnky

Downloading...
From: https://drive.google.com/uc?id=1vrfoukPD0GcoLaP8RewHvIKzcL-8Pnky
To: /content/German.xlsx
15.0MB [00:00, 35.9MB/s]


In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset,load_metric
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [4]:
df = pd.read_excel('/content/German.xlsx')
df.head()

,English,German
0,Resumption of the session,Wiederaufnahme der Sitzungsperiode
1,I declare resumed the session of the European ...,"Ich erkläre die am Freitag, dem 17. Dezember u..."
2,"Although, as you will have seen, the dreaded '...","Wie Sie feststellen konnten, ist der gefürchte..."
3,You have requested a debate on this subject in...,Im Parlament besteht der Wunsch nach einer Aus...
4,"In the meantime, I should like to observe a mi...",Heute möchte ich Sie bitten - das ist auch der...


In [5]:
df.isnull().sum()

English    423
German     162
dtype: int64

In [6]:
df.dropna(axis = 0,inplace = True)
df.reset_index(inplace = True)
df.drop('index',axis = 1,inplace=True)

In [7]:
pd.Series({c: df[c].map(lambda x: len(str(x))).max() for c in df}).sort_values(ascending = False)#truncation required

English    1871
German     1607
dtype: int64

In [8]:
train_df,val_df = train_test_split(df,test_size = 0.15,random_state = 42)

In [9]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [10]:
print(train_dataset)
print(val_dataset)

Dataset({
    features: ['English', 'German', '__index_level_0__'],
    num_rows: 84502
})
Dataset({
    features: ['English', 'German', '__index_level_0__'],
    num_rows: 14913
})


In [11]:
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
val_dataset = val_dataset.remove_columns(['__index_level_0__'])

In [12]:
train_dataset['English'][0]

'But is this the right level at which to produce such very detailed technical descriptions as this Directive is attempting to produce?'

In [13]:
checkpoint = "Helsinki-NLP/opus-mt-en-de"
from transformers import MarianConfig,MarianMTModel,MarianTokenizer
tokenizer = MarianTokenizer.from_pretrained(checkpoint)
model = MarianMTModel.from_pretrained(checkpoint)

In [14]:
tokenizer(train_dataset['English'][0:2])

{'input_ids': [[390, 19, 60, 4, 315, 575, 67, 86, 12, 4098, 243, 256, 3476, 1219, 17942, 52, 60, 830, 19, 35586, 12, 4098, 31, 0], [36, 3451, 19, 7, 780, 12, 1722, 2, 226, 8, 26349, 2, 14, 1323, 988, 1856, 86, 73, 2287, 4, 966, 5, 60, 405, 12, 43, 24435, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [15]:
for idx,i in enumerate(train_dataset['English']):
  print(i)
  if idx == 5:
    break

But is this the right level at which to produce such very detailed technical descriptions as this Directive is attempting to produce?
The objective is of course to create, first and foremost, a clear general framework which will allow the law in this area to be simplified.
I find it totally unacceptable that we are witnessing violations of international humanitarian law by the Russian army.
By renovating those structures and systems, in some cases very radically, we will assist the people of the Commission who are strongly committed to the European ideal and have high quality of capability to release talents and manifest themselves as never before.
It is, in fact, important that the candidate countries be involved.
Vaclav Havel stated that, in Eastern Europe, the Communist regimes methodically destroyed civil society: national movements, churches and the independent trade unions.


In [16]:
type(train_dataset['English'])

list

In [17]:
def preprocess(examples,MAX_LENGTH = 256):
  model_inputs = tokenizer(examples['English'],
                           max_length=MAX_LENGTH,
                           padding = 'max_length',
                           truncation=True,
                           return_attention_mask = True)
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(examples['German'],
                        max_length=MAX_LENGTH,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask=True)
  model_inputs['labels'] = targets['input_ids']
  model_inputs['decoder_input_ids'] = targets['input_ids']
  model_inputs['decoder_attention_mask'] = targets['attention_mask']
  return model_inputs

In [18]:
train_tokenized = train_dataset.map(preprocess,batched=True)
validation_tokenized = val_dataset.map(preprocess,batched=True)

In [19]:
train_tokenized = train_tokenized.remove_columns(['English','German'])
validation_tokenized = validation_tokenized.remove_columns(['English','German'])

In [20]:
# train_tokenized.set_format('torch')
# validation_tokenized.set_format('torch')

In [21]:
train_tokenized.column_names

['attention_mask',
 'decoder_attention_mask',
 'decoder_input_ids',
 'input_ids',
 'labels']

In [22]:
model.config.pad_token_id

58100

In [24]:
data_collator = DataCollatorForSeq2Seq(model = model,tokenizer = tokenizer,label_pad_token_id=-100)

In [27]:
metric = load_metric("sacrebleu")

In [28]:
batch_size = 16
learning_rate = 2e-5
args = Seq2SeqTrainingArguments(
    "test-translation",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub = True
)

In [29]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [30]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=validation_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [31]:
trainer.train()

***** Running training *****
  Num examples = 84502
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 26410
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.182500,0.170582,26.179800,30.513300
2,0.168400,0.169392,26.355900,30.438100
3,0.159500,0.169006,26.386700,30.488000
4,0.154600,0.169057,26.395500,30.404600
5,0.149000,0.169099,26.433300,30.399300


Saving model checkpoint to test-translation/checkpoint-500
Configuration saved in test-translation/checkpoint-500/config.json
Model weights saved in test-translation/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-translation/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-translation/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-translation/checkpoint-1000
Configuration saved in test-translation/checkpoint-1000/config.json
Model weights saved in test-translation/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-translation/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-translation/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-translation/checkpoint-1500
Configuration saved in test-translation/checkpoint-1500/config.json
Model weights saved in test-translation/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-translation/checkpo

TrainOutput(global_step=26410, training_loss=0.16361458318696606, metrics={'train_runtime': 19124.8101, 'train_samples_per_second': 22.092, 'train_steps_per_second': 1.381, 'total_flos': 9.658115685875712e+16, 'train_loss': 0.16361458318696606, 'epoch': 5.0})

In [66]:
def predict_output(english_text,german_text):
  token = tokenizer(english_text,
                    max_length=256,
                    padding = 'max_length',
                    truncation = True,
                    return_tensors = 'pt')
  with tokenizer.as_target_tokenizer():
    target_token = tokenizer(german_text,
                             max_length=256,
                             truncation = True,
                             padding = 'max_length',
                             return_tensors = 'pt')
  token.to('cuda')
  target_token.to('cuda')

  output = model(**token, 
                 decoder_input_ids = target_token['input_ids'],
                 decoder_attention_mask = target_token['attention_mask'])
  pred = torch.argmax(output.logits,dim = 2)
  for i in pred:
    x = tokenizer.decode(i,skip_special_tokens = True)
  return x

In [69]:
predict_output('The objective is of course to create, first and foremost, a clear general framework which will allow the law in this area to be simplified.',
               'Ziel ist es natürlich in erster Linie, einen klaren allgemeinen Rahmen zu schaffen, der eine Vereinfachung des Rechts in diesem Bereich ermöglicht.')

'Ziel ein,,, ein ein ein klaren allgemeinen Rahmen, schaffen, der die Reihe der Rechts in der Bereich ermöglicht.'